# Chapter 8: Data Wrangling - Join, Combine, and Reshape

In [1]:
import numpy as np
import pandas as pd
np.random.seed(42)

## 8.1 Hierarchical Indexing

*Hierarchical indexing* is an important feature of pandas that enables you to have multiple (two or more) index *levels* on an axis.

This is a prettified view of a Series with `MultiIndex` as its index. The "gaps" mean "use the label directly above."

In [19]:
data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.234587
   2   -1.415371
   3   -0.420645
b  1   -0.342715
   3   -0.802277
c  1   -0.161286
   2    0.404051
d  2    1.886186
   3    0.174578
dtype: float64

In [20]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

`Partial` indexing is possible, enabling selection of subsets of data.

In [21]:
data['b']

1   -0.342715
3   -0.802277
dtype: float64

In [22]:
data['b': 'c']

b  1   -0.342715
   3   -0.802277
c  1   -0.161286
   2    0.404051
dtype: float64

In [23]:
data.loc[['b', 'd']]

b  1   -0.342715
   3   -0.802277
d  2    1.886186
   3    0.174578
dtype: float64

In [24]:
data.loc[:, 2]

# First selects all rows, then 2

a   -1.415371
c    0.404051
d    1.886186
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group-based operations. 

For example, you could rearrange data into a DataFrame using `unstack()`.

In [25]:
data.unstack()

,1,2,3
a,-0.234587,-1.415371,-0.420645
b,-0.342715,NaN,-0.802277
c,-0.161286,0.404051,NaN
d,NaN,1.886186,0.174578


In [26]:
data.unstack().stack()

a  1   -0.234587
   2   -1.415371
   3   -0.420645
b  1   -0.342715
   3   -0.802277
c  1   -0.161286
   2    0.404051
d  2    1.886186
   3    0.174578
dtype: float64

With a DataFrame, either axis can have hierarchical index.

In [27]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                            ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [28]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

> Note: Be careful to distinguish the index names `'state'` and `'color'` from the row labels.

In [29]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

`MultiIndex` can be created by itself and then reused.

In [31]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'],
                            ['Green', 'Red', 'Green']],
                            names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### 8.1.1 Reordering and Sorting Levels

At times you will need to rearrange the order of the levels on an axis or sort the data by the values in one specific level.

`swaplevel` takes two level numbers or names and returns a new object with the levels interchanged.

In [32]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` sorts the data using only the values in a single level. It's also used to lexicographically sort at the indicated level.

In [33]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [34]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 8.1.2 Summary Statistics by Level

Many descriptive and summary statistics on DataFrame and Series have a `level` option in which you specify the level you want to aggregate by on a particular axis.

In [35]:
frame.sum(level='key2')

# Specifically in 'key2', sum up all the values in respective rows.

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [36]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 Indexing with a DataFrame's columns

It's not unusual to want to use one or more columns from a DataFrame as the row index.  
Or move the row index into a DataFrame column.

In [37]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                        'c': ['one', 'one', 'one', 'two', 'two',
                                'two', 'two'],
                        'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame's `set_index()` will create a new DataFrame using one or more of its columns as the index.

In [38]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [39]:
frame.set_index(['c', 'd'], drop=False)

# By default columns are removed
# Leave them in with 'drop=False'

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

Use `reset_index()` to move the hierarchical index levels into the columns.

In [40]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 Combining and Merging Datasets

- `pandas.merge` connects rows in DataFrames based on one or more keys.
- `pandas.concat` concatenates or "stacks" together objects along an axis.
- `combine_first` enables splicing together overlapping data to fill in missing values in one object with values from another.

### 8.2.1 Database-Style DataFrame Joins

In [2]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [3]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


*Many-to-one join* -> df1 has multiple rows labeled a and b, df2 has only one row for each value in the key column.

In [4]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


> Note: If which column to join on is not specified, `merge` uses the overlapping column names as the keys.

In [5]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


If the column names are different in each object, you can specify separately.

In [6]:
df3 = pd.DataFrame({'lkey': ['b' ,'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


> Note: By default `merge` does an `'inner'` join (intersection, or common set found in both tables). `'outer'` takes the union of the keys (combining both `'left'` and `'right'`).

In [7]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


*Many-to-many* merges are not so intuitive.

In [8]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data1': range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [9]:
df2

,key,data1
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [10]:
pd.merge(df1, df2, on='key', how='left')

,key,data1_x,data1_y
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


Many-to-many joins form the Cartesian product of the rows.

In [11]:
pd.merge(df1, df2, how='inner')

,key,data1
0,b,1
1,a,2


Pass a list of column names to merge with multiple keys.

In [14]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                    'key2': ['one', 'two', 'one'],
                    'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                    'key2': ['one', 'one', 'one', 'two'],
                    'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


`merge` has `suffixes` option for specifying strings to append to overlapping names in the left and right DataFrame objects.

In [15]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [16]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 8.2.2 Merging on Index

Pass `left_index=True` or `right_index=True` (or both) to indicate that the index should be used as a merge key.

In [17]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                        'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [20]:
right1

,group_val
a,3.5
b,7.0


In [22]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [23]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Hierarchical indexed data is implicitly a multiple-key merge.

In [27]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                                'Nevada', 'Nevada'],
                        'key2': [2000, 2001, 2002, 2001, 2002],
                        'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                        index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                                'Ohio', 'Ohio'],
                                [2001, 2000, 2000, 2000, 2001, 2002]],
                                columns=['event1', 'event2'])

lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [28]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

You have to indicate multiple columns to merge on as a list.

> Note: The handling of duplicate values with `how='outer'`.

In [29]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [30]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
        right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


Using indexes of both sides of the merge is also possible.

In [31]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                        index=['a', 'c', 'e'],
                        columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                        index=['b', 'c', 'd', 'e'],
                        columns=['Missouri', 'Alabama'])

left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [32]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [33]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame has `join` for merging by index, and can be used to combine together many DataFrame objects having the same or similar indexes but non-overlapping columns.

In [34]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [35]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


For simple index-on-index merges, pass a list of DataFrames to `join` as an alternative to using the more general `concat`.

In [36]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                        index=['a', 'c', 'e', 'f'],
                        columns=['New York', 'Oregon'])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [37]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [38]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### 8.2.3 Concatenating Along an Axis

NumPy's `concatenate` can concatenate/bind/stack NumPy arrays.

In [39]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [42]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

Suppose we have three Series with no index overlap.

In [43]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default, `concat` works along `axis=0` producing another Series. Passing `axis=1` will instead result in a DataFrame (`axis=1` is the columns).

In [44]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [45]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [46]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [47]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


~~`join_axes` specifies which axes to be used on the other axes.~~

> **WARNING:**  
> ** `join_axes=()` has been depreciated**  
> **Instead use `.reindex()` before/after concatenation**

In [60]:
pd.concat([s1, s4], axis=1, join='inner').reindex(['a', 'c', 'b', 'e'])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


Use the `keys` argument to create a hierarchical index on the concatenation axis. 

In [61]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [62]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [63]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [64]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                    columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                    columns=['three', 'four'])

df1

,one,two
a,0,1
b,2,3
c,4,5


In [65]:
df2

,three,four
a,5,6
c,7,8


In [66]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

If a dict of objects is passed instead of a list, then the dict's keys will be used for the keys option.

In [67]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [68]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
            names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

For row indices that do not contain any relevant data.

In [70]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

df1

,a,b,c,d
0,-0.908024,-1.412304,1.465649,-0.225776
1,0.067528,-1.424748,-0.544383,0.110923
2,-1.150994,0.375698,-0.600639,-0.291694


In [71]:
df2

,b,d,a
0,-0.601707,1.852278,-0.013497
1,-1.057711,0.822545,-1.220844


In [72]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.908024,-1.412304,1.465649,-0.225776
1,0.067528,-1.424748,-0.544383,0.110923
2,-1.150994,0.375698,-0.600639,-0.291694
3,-0.013497,-0.601707,NaN,1.852278
4,-1.220844,-1.057711,NaN,0.822545


### 8.2.4 Combining Data with Overlap

You may have two datasets whose indexes overlap in full or part.  

Consider NumPy's `where` function, which performs array-oriented equivalent of an if-else expression.

In [73]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
                index=['f', 'e', 'd' ,'c', 'b', 'a'])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
                index=['a', 'b', 'c', 'd', 'e', 'f'])

a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [74]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [75]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [76]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

DataFrame's `combine_first` does the same thing column by column.

> Note: Think of it as "patching" missing data in the calling object with data from the object you pass.

In [77]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [78]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [79]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3 Reshaping and Pivoting

### 8.3.1 Reshaping with Hierarchical Indexing

### 8.3.2 Pivoting "Long" to "Wide" Format

### 8.3.3 Pivoting "Wide" to "Long" Format

## 8.4 Conclusion